Note Description:

- need two inputs: time series of S1 images and landscape polygons

- need to determine the best landscape segmentation product

- 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import glob
import geopandas as gpd

import xarray as xr
import rioxarray as rxr

from rasterio.features import geometry_mask

share_folder = '/geoanalytics_user_shared_data/vtang/PEOPLE-ER_Vietnam/'
# share_folder = '/home/jovyan/geoanalytics_user_shared_data/vtang/PEOPLE-ER_Vietnam/'

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-bgtqi_if because the default path (/home/users/vtang/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Load Sentinel-1 Data

- stack time series images (composited at different scale)

In [2]:
target_year = 2022
target_nday = 10

fpath_list = glob.glob(
    share_folder + 'sentinel-1/{}/{}d_composition/*.tif'.format(target_year, target_nday))

da_list = [rxr.open_rasterio(fpath) for fpath in fpath_list]

# da_list = [
#     da.chunk({'band': -1, 'x': 128, 'y':128})
#     for da in da_list
# ]

x = [int(item.split('_')[-1].split('.')[0]) for item in fpath_list]

In [3]:
da = xr.concat(da_list, pd.Index(x))
da.values = 10 * np.log10(da.values)

# Load Landscape Segmentation

In [4]:
# gdf = gpd.read_file('../data/study_area/rice_crop_fields.shp')
gdf = gpd.read_file('../data/study_area/rice_crop_fields_2.geojson')

gdf = gdf.to_crs(da.rio.crs.to_epsg())
gdf = gdf.sort_values('Type')

gdf = gdf.set_index('id')

# Generate Time Series

In [5]:
data_dict = {}
count_list = []

for idx, row in gdf.iterrows():
    
    xmin, ymin, xmax, ymax = row.geometry.bounds
    da2 = da.sel(x=slice(xmin, xmax), y=slice(ymax, ymin))

    mask = geometry_mask(
        gpd.GeoSeries([row.geometry]),
        out_shape=da2.shape[-2:],
        transform=da2.rio.transform(),
    )
    mask = ~mask

    data = da2.values[:, :, mask]
    data = np.mean(data, axis=2)
    data_dict.update({idx: data})

    count = np.sum(mask)
    count_list.append(count)

gdf['# Pixels'] = count_list

In [6]:
idx, mat = [], []
for i, val in data_dict.items():
    idx.append(i)
    mat.append(val[:, 0])   
mat = np.vstack(mat)
df_vh = pd.DataFrame(mat, index=idx, columns=x)
df_vh.columns = pd.MultiIndex.from_product([['VH'], df_vh.columns])

idx, mat = [], []
for i, val in data_dict.items():
    idx.append(i)
    mat.append(val[:, 1]) 
mat = np.vstack(mat)
df_vv = pd.DataFrame(mat, index=idx, columns=x)
df_vv.columns = pd.MultiIndex.from_product([['VV'], df_vv.columns])

df_attr = gdf[['Type', '# Pixels']]
df_attr.columns = pd.MultiIndex.from_product([['Attr'], ['Type', '# Pixels']])

df = pd.concat([df_attr, df_vv, df_vh], axis=1)
df.to_csv('../data/crop_time_series/{}_{}d.csv'.format(target_year, target_nday))